# Summary

## Demonstration of how to pull results from an mlflow repository

mlflow results are stored as runs nested in experiments and can be pulled individually and as a group.  The following is an example of the interactions.

Shown includes:
* searching across experiments
* excluding based on filters
* setting runs to finished
* deleting runs or experiments and searching based on that status

In [1]:
import mlflow
import mlflow.tracking
from mlflow.tracking import MlflowClient
import os
import shutil
from pprint import pprint

In [2]:
# Enumerators from mlflow

# ALL shows active and deleted rows.  
ALL = mlflow.entities.view_type.ViewType.ALL

# ACTIVE_ONLY shows only runs that were not 'deleted'
ACTIVE_ONLY = mlflow.entities.view_type.ViewType.ACTIVE_ONLY

# FINISHED shows anything that was marked as "terminated" with FINISHED as the code
FINISHED = mlflow.entities.RunStatus.to_string(mlflow.entities.RunStatus.FINISHED)

### Create a mlflow repo to interact with

In [3]:
temp_mlflow_uri = "./temp_mlflow/"

if os.path.exists(temp_mlflow_uri) and os.path.isdir(temp_mlflow_uri):
    shutil.rmtree(temp_mlflow_uri)

client = MlflowClient(temp_mlflow_uri)

### Create 3 experiments with 5 runs each and 10 internal steps 

Odd numbered runs are set to finished (2 per experiment)
Last run in each experiment is deleted
Experiment 2 is deleted entirely

In [4]:
n_experiments = 3
n_runs = 5
n_steps = 10
SOME_TAG_AT_RUN_CREATION = 'tag_name'
TAG_VALUE = 'value'
METRIC_NAME = "some_metric"

for i_experiment in range(n_experiments):
    this_exp_id = client.create_experiment(f"Experiment {i_experiment}")
    for i_run in range(n_runs):
        this_run = client.create_run(this_exp_id, tags={SOME_TAG_AT_RUN_CREATION: TAG_VALUE})
        run_id = this_run.info.run_id
        client.log_param(run_id, "some_param", i_run)
        for i_step in range(n_steps):
            client.log_metric(run_id, METRIC_NAME, i_run / 5. * i_step / n_steps, step=i_step)
        # Set odd runs to finished
        if i_run % 2 == 1:
            client.set_terminated(run_id)
            
        # Delete the first run
        if i_run == 0: 
            client.delete_run(run_id)
            
client.delete_experiment(this_exp_id)

### Get the history of a metric for a single run

In [5]:
exp = client.list_experiments()[0]
# print(exp)
run_infos = client.list_run_infos(experiment_id=exp.experiment_id)
# print(run_infos)
run_id = run_infos[0].run_id
metric_history = client.get_metric_history(run_id=run_id, key=METRIC_NAME)
pprint(metric_history)

[<Metric: key='some_metric', step=0, timestamp=1583090423573, value=0.0>,
 <Metric: key='some_metric', step=1, timestamp=1583090423575, value=0.08>,
 <Metric: key='some_metric', step=2, timestamp=1583090423577, value=0.16>,
 <Metric: key='some_metric', step=3, timestamp=1583090423579, value=0.24000000000000005>,
 <Metric: key='some_metric', step=4, timestamp=1583090423581, value=0.32>,
 <Metric: key='some_metric', step=5, timestamp=1583090423583, value=0.4>,
 <Metric: key='some_metric', step=6, timestamp=1583090423585, value=0.4800000000000001>,
 <Metric: key='some_metric', step=7, timestamp=1583090423588, value=0.56>,
 <Metric: key='some_metric', step=8, timestamp=1583090423590, value=0.64>,
 <Metric: key='some_metric', step=9, timestamp=1583090423592, value=0.72>]


### Search for the best metric across multiple experiments

Searches all active (not deleted) experiments, all runs (active and deleted).  Should find 10

In [6]:
experiments = client.list_experiments()  # by default searches ALL_ACTIVE
experiment_ids = [exp.experiment_id for exp in experiments]

runs = client.search_runs(experiment_ids=experiment_ids, 
#                            filter_string=filter_string,
                           # Filter by the active/deleted built-in metadata (lifecycle_stage)
                           # (can use for deprecating past runs)
                           run_view_type=ALL,

                           # Order using metrics, params, etc
                           order_by=[f'metric.{METRIC_NAME}'],
                          )

print(f"Found {len(runs)} runs")
print("Showing first two runs returned:")
pprint(runs[:2])

Found 10 runs
Showing first two runs returned:
[<Run: data=<RunData: metrics={'some_metric': 0.0}, params={'some_param': '0'}, tags={'tag_name': 'value'}>, info=<RunInfo: artifact_uri='./temp_mlflow/2/5d2c4c31bc9f4d7889becb056c33a36f/artifacts', end_time=None, experiment_id='2', lifecycle_stage='deleted', run_id='5d2c4c31bc9f4d7889becb056c33a36f', run_uuid='5d2c4c31bc9f4d7889becb056c33a36f', start_time=1583090423598, status='RUNNING', user_id='unknown'>>,
 <Run: data=<RunData: metrics={'some_metric': 0.0}, params={'some_param': '0'}, tags={'tag_name': 'value'}>, info=<RunInfo: artifact_uri='./temp_mlflow/1/7da8066a588b4121a8ecd9e3914f052d/artifacts', end_time=None, experiment_id='1', lifecycle_stage='deleted', run_id='7da8066a588b4121a8ecd9e3914f052d', run_uuid='7da8066a588b4121a8ecd9e3914f052d', start_time=1583090423427, status='RUNNING', user_id='unknown'>>]


### Or search only "finished" runs (terminated)

Searches ACTIVE_ONLY experiments. Should find 4

In [7]:
# attributes.status is set using the termination function above (can be more than just finished and running)
filter_string = f"attributes.status = '{FINISHED}'"# AND {SOME_TAG_AT_RUN_CREATION} = '{TAG_VALUE}'" # Only with certain tag (could use this as data version, could also be a param value)

experiments = client.list_experiments()
experiment_ids = [exp.experiment_id for exp in experiments]

runs = client.search_runs(experiment_ids=experiment_ids, 
                           filter_string=filter_string,
                           # Filter by the active/deleted built-in metadata (lifecycle_stage)
                           # (can use for deprecating past runs)
                           run_view_type=ALL,

                           # Order using metrics, params, etc
                           order_by=[f'metric.{METRIC_NAME}'],
                          )

print(f"Found {len(runs)} runs")
# pprint(runs)

Found 4 runs


### Or search only "finished" runs (terminated) that have a tag

Searches ACTIVE_ONLY experiments.  Should find 4

In [8]:
# Could use tag filters for data_version == relevant_data, or model_version == relevant_model
# Model versioning also supported within mlflow, but not sure if it is good?
filter_string = f"attributes.status = '{FINISHED}' and tag.{SOME_TAG_AT_RUN_CREATION} = '{TAG_VALUE}'" # Only with certain tag (could use this as data version, could also be a param value)

experiments = client.list_experiments()
experiment_ids = [exp.experiment_id for exp in experiments]

runs = client.search_runs(experiment_ids=experiment_ids, 
                           filter_string=filter_string,
                           # Filter by the active/deleted built-in metadata (lifecycle_stage)
                           # (can use for deprecating past runs)
                           run_view_type=ALL,

                           # Order using metrics, params, etc
                           order_by=[f'metric.{METRIC_NAME}'],
                          )

print(f"Found {len(runs)} runs")
# pprint(runs)

Found 4 runs


### Search from ALL experiments (active or deleted) to get any "finished" runs that are not deleted (run is not deleted, but it could be in a deleted experiment)

Should find 6

In [9]:
# Create a string filter that combines multiple criteria
filter_string = f"attributes.status = '{FINISHED}'"

experiments = client.list_experiments(view_type=ALL)
experiment_ids = [exp.experiment_id for exp in experiments]

runs = client.search_runs(experiment_ids=experiment_ids, 
                           filter_string=filter_string,
                           # Filter by the active/deleted built-in metadata (lifecycle_stage)
                           # (can use for deprecating past runs)
                           run_view_type=ACTIVE_ONLY,

                           # Order using metrics, params, etc
                           order_by=[f'metric.{METRIC_NAME}'],
                          )

print(f"Found {len(runs)} runs")
# pprint(runs)

Found 6 runs


## Pull the best finished run from all active experiments/runs

In [10]:
# Create a string filter that combines multiple criteria
filter_string = f"attributes.status = '{FINISHED}'"

experiments = client.list_experiments(view_type=ACTIVE_ONLY)
experiment_ids = [exp.experiment_id for exp in experiments]

runs = client.search_runs(experiment_ids=experiment_ids, 
                           filter_string=filter_string,
                           # Filter by the active/deleted built-in metadata (lifecycle_stage)
                           # (can use for deprecating past runs)
                           run_view_type=ACTIVE_ONLY,

                           # Order using metrics, params, etc
                           order_by=[f'metric.{METRIC_NAME}.DESC'],
                          )

print(f"Found {len(runs)} runs")
best_run = runs[0]
print("params of best run:")
pprint(best_run.data.params)

Found 4 runs
params of best run:
{'some_param': '3'}


# Cleanup

In [11]:
if os.path.exists(temp_mlflow_uri) and os.path.isdir(temp_mlflow_uri):
    shutil.rmtree(temp_mlflow_uri)